## Language model for error row finder

In [208]:
from __future__ import generator_stop
import numpy as np
import random
import nltk, re, string, collections
from nltk.util import ngrams
import pandas as pd
from nltk.probability import *
import string

li = ['mailid']
temp = pd.read_csv("./dataset.csv")
temp = temp.fillna("")
tempn = pd.DataFrame()
for i in temp.columns:
    tempn[str(i)] = '*' + temp[str(i)].map(str) + '*'
npdf = tempn[tempn['output'] =="*0*"]


def map_parameter_init():
    d = {}
    pun = list(string.punctuation)
    num = list(string.digits)
    let = list(string.ascii_letters)
    for i in pun:
        d[i] = i
    for j in num:
        d[j] = 'n'
    for k in let:
        d[k] = 'c'
    d[' '] = ' '
    d['\n'] = '\n'
    return d

d = map_parameter_init()
vi = dict()
cfd = dict()
rejli = []
rejrli = []
rejoli = []
rejpd = pd.DataFrame().reindex_like(temp)
mingram = dict()

In [209]:
def mapval(a):
    u,inv = np.unique(a,return_inverse = True)
    te = tuple(np.array([d[x] for x in u])[inv])
    lin.append(te)
    
for i in temp.columns:
    #distribution counting
    lin = []
    tokd = pd.DataFrame(npdf[str(i)].map(str).apply(list))
    val = tokd[str(i)].apply(mapval)
    vi[str(i)] = collections.Counter(lin)
    
    ##Language model implementation
    toke = list(npdf[str(i)].str.cat(sep='\n'))
    minv = npdf[str(i)].map(len).min()
    mingram[str(i)] = minv
    u,inv = np.unique(toke,return_inverse = True)
    te = np.array([d[x] for x in u])[inv].reshape(len(toke))
    total = len(te)
    bigr = nltk.ngrams(te,mingram[str(i)])
    condition_pairs = (((w[:-1]), w[-1]) for w in bigr)
    cfd[str(i)] = nltk.ConditionalFreqDist(condition_pairs)
print(cfd)

{'timestamp': <ConditionalFreqDist with 17 conditions>, 'description': <ConditionalFreqDist with 1569 conditions>, 'ipaddress': <ConditionalFreqDist with 159 conditions>, 'limit6': <ConditionalFreqDist with 336 conditions>, 'float': <ConditionalFreqDist with 28 conditions>, 'year': <ConditionalFreqDist with 7 conditions>, 'money': <ConditionalFreqDist with 15 conditions>, 'country': <ConditionalFreqDist with 17 conditions>, 'Enum': <ConditionalFreqDist with 5 conditions>, 'mailid': <ConditionalFreqDist with 860 conditions>, 'bool': <ConditionalFreqDist with 8 conditions>, 'longnumber': <ConditionalFreqDist with 11 conditions>, 'stockprice': <ConditionalFreqDist with 20 conditions>, 'output': <ConditionalFreqDist with 4 conditions>}


In [210]:
#run on actual data
def tmapval(a, va,i):
    u,inv = np.unique(a,return_inverse = True)
    te = tuple(np.array([d[x] for x in u])[inv])
    try:
        tel = vi[str(i)].get(te)/va
    except:
#         print(a)
        rejli.append(te)
        rejrli.append(a)
        
def feature_mapping():
    count = 0
    for i in li:
        tdf = pd.DataFrame(tempn[str(i)].map(str).apply(list))
        count = len(tdf.index)
        tempa = tdf[str(i)].apply(tmapval, args = (count,i))
        print("Length of rejected items:", len(rejli),"\n")
        if rejli !=[]:
            for j in rejli:
                tr = nltk.ngrams(j,mingram[str(i)])
                if list(tr) == []:
                    vi = 0
                    print("empty reject:","".join(j))
                    continue
                else:
                    tr = nltk.ngrams(j,mingram[str(i)])
                    cpairs = (((w[:-1]), w[-1]) for w in tr)
                    co = 0
                    vi = 0
                    for k in list(cpairs):
                        v = cfd[str(i)][k[0]][k[1]]
                        if co == 0:
                            vi = v/total
                            co = 1
                        else:
                            vi *= v/total
                    if vi ==0:
                        print("non empty, zero prob so reject","".join(rejrli[rejli.index(j)]))
                    elif vi!=0:
                        print("non empyt, not zero so accepted","".join(rejrli[rejli.index(j)]))
                
        
x = feature_mapping()

Length of rejected items: 150 

non empty, zero prob so reject *@sdockerty4jmapquest.com*
non empty, zero prob so reject *msolleym1@vk.com*
non empyt, not zero so accepted *plaguerre6v@imdb.com*
non empyt, not zero so accepted *acausnett8a@tumblr.com*
non empyt, not zero so accepted *kmereweather68@addthis.com*
non empyt, not zero so accepted *vgwinnel5v@topsy.com*
non empyt, not zero so accepted *rcartmel1z@telegraph.co.uk*
non empyt, not zero so accepted *ctapin13@webnode.com*
non empyt, not zero so accepted *ttroy3v@comsenz.com*
non empty, zero prob so reject *amcgeechan1r%vinaora.com*
non empyt, not zero so accepted *backenheadah@nba.com*
non empyt, not zero so accepted *gdrakeford5y@dion.ne.jp*
non empyt, not zero so accepted *rcartmel1z@telegraph.co.uk*
empty reject: **
non empyt, not zero so accepted *gcuzenep@admin.ch*
non empyt, not zero so accepted *jcoole5x@liveinternet.ru*
non empyt, not zero so accepted *lhovy3k@slideshare.net*
non empyt, not zero so accepted *frodd32@dede

C:\Users\gsoun\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:22: DeprecationWarning: generator 'ngrams' raised StopIteration


In [171]:
cfd['ipaddress'][('n', '.', 'n', 'n', 'n', '.', 'n', 'n', 'n', '.')]

FreqDist({'n': 234})

18

In [104]:
# condition_pairs.throw()
# a = {}
# k = 0
# while k < 10:
#     <dynamically create key> 
#     key = ...
#     <calculate value> 
#     value = ...
#     a[key] = value 
#     k += 1

# t = nltk.ngrams("timestamp,description,ipaddress,limit6,float,year,money,country,Enum,mailid,bool,longnumber,stockprice,output", 3)
# # list(t)
# for i in list(t):
#     print(i)
#     print(i[:-1])
#     print(i[-1])
    
# t = nltk.ngrams("timestamp,description,ipalongnumber,stockprice,output", 3)
# if list(t) == []:
#     print("in")
# else:
#     print("out")

    
# # collections.Counter(vi.most_common(1))
# l = list(vi.keys())
# for i in l:
#     print(collections.Counter(i))
# npdf['mailid'].map(len).sum()/len(npdf['mailid'].index) $%average length of list in a columnx
# fdist = nltk.FreqDist(w.lower() for w in te)
# total = 0
# for word in fdist:
#     total += fdist[word]
    
# v = list('**nncn')
# for w in v:
#     if w in fdist:
#         print(w,'freq',fdist[w],'prob',fdist[w]/total)
#     else:
#         print(w,'freq',0,'prob',0)
# cfd.items()
# cfd.keys()
# cfd[v[0]][v[1]]

# toke = list(data)
# u,inv = np.unique(toke,return_inverse = True)
# te = np.array([d[x] for x in u])[inv].reshape(len(toke))
# esBigrams = ngrams(te, 3)
# # list(esBigrams)[:10]
# esBigramFreq = collections.Counter(esBigrams)
# esBigramFreq.most_common(20)
